# 1.0 Extract movement and audio

For each video we extract movement data with YOLOv8 and audio data with moviepy/ffmpeg.


# 1.1 Video pose estimation with YOLOv8

[YOLOv8](https://github.com/ultralytics/ultralytics) is a commercially maintained version of the YOLO object recognition model. [Yolov7](https://github.com/WongKinYiu/yolov7) introduced pose estimation and v8 improves the models and makes everything much more user-friendly. It can be installed as a package

* Pip : `pip install ultralytics`
* Conda : `conda install -c conda-forge ultralytics`


In [2]:
import os
import math
import sys
import time
import pandas as pd
import numpy as np
import cv2
import torch
from ultralytics import YOLO

In [3]:
videos_in = "..\\LookitLaughter.test\\"
videos_out = "..\\data\\1_interim\\"

#get metadata from excel file
metadata = pd.read_excel(videos_in+ "_LookitLaughter.xlsx")
metadata.head()

,Unnamed: 0,VideoID,Consent,Sharing,ParentID,ChildID,Age,Gender,PregnancyLength,Languages,...,UseOfMedia,Withdrawal,DatabraryShare,JokeType,HowFunny,LaughYesNo,JokeNum,JokeRep,JokeTake,shortname
0,0,2UWdXP.joke1.rep2.take1.Peekaboo.mp4,accepted,public,5ZQUSU,2UWdXP,233,f,38 weeks,en,...,public,False,yes,Peekaboo,Slightly funny,No,1,2,1,NaN
1,1,2UWdXP.joke1.rep3.take1.Peekaboo.mp4,accepted,public,5ZQUSU,2UWdXP,233,f,38 weeks,en,...,public,False,yes,Peekaboo,Slightly funny,No,1,3,1,NaN
2,2,2UWdXP.joke2.rep1.take1.NomNomNom.mp4,accepted,public,5ZQUSU,2UWdXP,233,f,38 weeks,en,...,public,False,yes,NomNomNom,Funny,No,2,1,1,NaN
3,3,2UWdXP.joke2.rep2.take1.NomNomNom.mp4,accepted,public,5ZQUSU,2UWdXP,233,f,38 weeks,en,...,public,False,yes,NomNomNom,Slightly funny,No,2,2,1,NaN
4,4,2UWdXP.joke2.rep3.take1.NomNomNom.mp4,accepted,public,5ZQUSU,2UWdXP,233,f,38 weeks,en,...,public,False,yes,NomNomNom,Slightly funny,No,2,3,1,NaN


In [32]:
#get yolo model with pose estimation
model = YOLO('yolov8n-pose.pt')

In [9]:
#helper functions
import torch
import pandas as pd

def getprocessedvideos(videos_out):
    #check if we have already processed some videos
    if os.path.exists(videos_out + "\\processedvideos.xlsx"):
        print("found existing processedvideos.xlsx")
        processedvideos = pd.read_excel(videos_out + "\\processedvideos.xlsx")
    else:
        #create new dataframe for info about processed videos
        print("creating new processedvideos.xlsx")
        cols = ["VideoID","ChildID", "JokeType","JokeNum","JokeRep","JokeTake", "HowFunny","LaughYesNo", "Frames", "FPS", "Width", "Height", "Duration","Keypoints.when", "Keypoints.file","Audio.when","Audio.file","Faces.when","Faces.file","Speech.when","Speech.file","LastError"]
        processedvideos = pd.DataFrame(columns=cols)
    return processedvideos

def createkeypointsdf():
    #create empty dataframe to store keypoints, one per person per frame
    bodyparts = ['nose', 'left_eye', 'right_eye', 'left_ear', 'right_ear', 'left_shoulder', 'right_shoulder', 'left_elbow', 'right_elbow', 'left_wrist', 'right_wrist', 'left_hip', 'right_hip', 'left_knee', 'right_knee', 'left_ankle', 'right_ankle']
    coords = ['x', 'y']
    conf = ['c']
    bodypartsxy = [f"{bp}.{c}" for bp in bodyparts for c in coords]
    bodypartsc = [f"{bp}.{c}" for bp in bodyparts for c in conf]
    boundingbox = [ 'bboxcent.x', 'bboxcent.y', 'bbox.width', 'bbox.height', 'bbox.c' ]
    cols = ['frame', 'person'] + boundingbox + bodypartsxy + bodypartsc
    df = pd.DataFrame(columns=cols)
    return df

def addkeypointstodf(df, framenumber, bbox,bconf, keypoints, kconf):
    for idx in range(len(bbox)):
        row = [framenumber, idx]
        row += torch.flatten(bbox[idx]).tolist()
        row += torch.flatten(bconf[idx]).tolist()
        row += torch.flatten(keypoints[idx]).tolist()
        row += torch.flatten(kconf[idx]).tolist()
        #add row to dataframe
        #print(row)
        df.loc[len(df)] = row
    return df

def videotokeypoints(videopath):
    # Run inference on the source
    results = model(videopath, stream=True)  # generator of Results objects
    df = createkeypointsdf()

    frame = 0
    for r in results:
        #print(torch.flatten(r.keypoints.xy[0]).tolist())
        df = addkeypointstodf(df,frame,r.boxes.xywh,r.boxes.conf,r.keypoints.xy, r.keypoints.conf)  
        frame += 1

    return df


In [10]:
processedvideos = getprocessedvideos(videos_out)
processedvideos.head()

found existing processedvideos.xlsx


,Unnamed: 0.1,Unnamed: 0,VideoID,ChildID,JokeType,JokeNum,JokeRep,JokeTake,HowFunny,LaughYesNo,...,Duration,Keypoints.when,Keypoints.file,Audio.when,Audio.file,Cleaned.when,Cleaned.file,LastError,Speech.file,Speech.when
0,0,0,2UWdXP.joke1.rep2.take1.Peekaboo.mp4,2UWdXP,Peekaboo,1,2,1,Slightly funny,No,...,15.175982,2023-09-19 11:39:38,..\data\1_interim\\2UWdXP.joke1.rep2.take1.Pee...,2023-09-19 12:31:14,..\data\1_interim\\2UWdXP.joke1.rep2.take1.Pee...,NaN,NaN,NaN,..\data\1_interim\2UWdXP.joke1.rep2.take1.Peek...,2023-09-20 16:58:38
1,1,1,2UWdXP.joke1.rep3.take1.Peekaboo.mp4,2UWdXP,Peekaboo,1,3,1,Slightly funny,No,...,10.585630,2023-09-19 11:40:12,..\data\1_interim\\2UWdXP.joke1.rep3.take1.Pee...,2023-09-19 12:31:15,..\data\1_interim\\2UWdXP.joke1.rep3.take1.Pee...,NaN,NaN,NaN,..\data\1_interim\2UWdXP.joke1.rep3.take1.Peek...,2023-09-20 16:58:39
2,2,2,2UWdXP.joke2.rep1.take1.NomNomNom.mp4,2UWdXP,NomNomNom,2,1,1,Funny,No,...,7.174514,2023-09-19 11:40:36,..\data\1_interim\\2UWdXP.joke2.rep1.take1.Nom...,2023-09-19 12:31:16,..\data\1_interim\\2UWdXP.joke2.rep1.take1.Nom...,NaN,NaN,NaN,..\data\1_interim\2UWdXP.joke2.rep1.take1.NomN...,2023-09-20 16:58:40
3,3,3,2UWdXP.joke2.rep2.take1.NomNomNom.mp4,2UWdXP,NomNomNom,2,2,1,Slightly funny,No,...,6.824348,2023-09-19 11:41:00,..\data\1_interim\\2UWdXP.joke2.rep2.take1.Nom...,2023-09-19 12:31:16,..\data\1_interim\\2UWdXP.joke2.rep2.take1.Nom...,NaN,NaN,NaN,..\data\1_interim\2UWdXP.joke2.rep2.take1.NomN...,2023-09-20 16:58:40
4,4,4,2UWdXP.joke2.rep3.take1.NomNomNom.mp4,2UWdXP,NomNomNom,2,3,1,Slightly funny,No,...,9.350991,2023-09-19 11:41:34,..\data\1_interim\\2UWdXP.joke2.rep3.take1.Nom...,2023-09-19 12:31:17,..\data\1_interim\\2UWdXP.joke2.rep3.take1.Nom...,NaN,NaN,NaN,..\data\1_interim\2UWdXP.joke2.rep3.take1.NomN...,2023-09-20 16:58:48


In [ ]:
#loop through each row of metadata and
#process all related videos
forcemetadata = False
forceprocess = False

for index, mrow in metadata.iterrows():
    #get VIDEOID from first column of metadata
    videoname = mrow["VideoID"]
    stemname = os.path.splitext(videoname)[0]
    print(f"video:{videoname}")

    #check we want to refill metadata or this video is not already in processedvideos dataframe
    if forcemetadata or videoname not in processedvideos["VideoID"].values: 
        #use cv2 to get fps and other video info to add to dataframe
        cap = cv2.VideoCapture(videos_in + videoname)    
        if (cap.isOpened()== False):
            print("Error opening video stream or file")
            continue
        else:
            #add row to processedvideos dataframe
            row = {"VideoID":videoname,
                "ChildID":mrow["ChildID"],
                "JokeType":mrow["JokeType"],
                "JokeNum":mrow["JokeNum"],
                "JokeRep":mrow["JokeRep"],
                "JokeTake":mrow["JokeTake"],
                "HowFunny":mrow["HowFunny"],
                "LaughYesNo":mrow["LaughYesNo"],
                "Frames":cap.get(cv2.CAP_PROP_FRAME_COUNT),
                "FPS":cap.get(cv2.CAP_PROP_FPS) , 
                "Width":cap.get(cv2.CAP_PROP_FRAME_WIDTH), 
                "Height":cap.get(cv2.CAP_PROP_FRAME_HEIGHT), 
                "Duration":cap.get(cv2.CAP_PROP_FRAME_COUNT)/cap.get(cv2.CAP_PROP_FPS)
                }
            cap.release()
            print(f"Adding video info: {row}")
            newrow = pd.DataFrame(row, index=[0])
            processedvideos = pd.concat([processedvideos,newrow], ignore_index=True)

    #select the dataframe row for this video 
    row = processedvideos.loc[processedvideos["VideoID"] == videoname]
    #check if we have already processed this video is keypoints is not nan
    if row.empty:
        print(f"error: no row for {videoname}")
        continue
    elif not forceprocess and not pd.isnull(row["Keypoints.file"].values[0]):
        print(row["Keypoints.file"].values[0]  )
        print(f"already processed {videoname}")
        continue
    else:
        #use ultralytics YOLO to get keypoints
        keypointsdf =videotokeypoints(videos_in + videoname)
        #save keypointsdf as csv    
        keypointspath = videos_out + "\\" + stemname + ".csv"
        keypointsdf.to_csv(keypointspath)
        row["Keypoints.file"] = keypointspath
        row["Keypoints.when"] = time.strftime("%Y-%m-%d %H:%M:%S", time.gmtime())
        #update this row in processedvideos dataframe
        processedvideos.loc[processedvideos["VideoID"] == videoname] = row
    

    #update processedvideos excel file
    processedvideos.to_excel(videos_out + "\\processedvideos.xlsx", index=False)

# 1.2 Audio extraction with moviepy

This step is a lot simpler. We extract the audio from each video and save it as `mp3`. We will use the `moviepy` library to do this. 
This will be helpful for later analysis and regenerating labeled videos with audio.

Note that `moviepy` is a wrapper around `ffmpeg` and `ffmpeg` needs to be installed separately. 

`conda install ffmpeg moviepy`

In [4]:
from moviepy.editor import VideoFileClip

In [11]:
def convert_video_to_audio_moviepy(videos_in, video_file, out_path, output_ext="mp3"):
    """Converts video to audio using MoviePy library
    that uses `ffmpeg` under the hood"""
    try:
        filename = os.path.splitext(video_file)[0]
        clip = VideoFileClip(videos_in + video_file)
        audio_file = f"{out_path}\\{filename}.{output_ext}"
        clip.audio.write_audiofile(audio_file)
        clip.close()
        return audio_file
    except Exception as e:
        print(f"Error converting {video_file} to {output_ext}: {e}")
        return None

In [12]:
processedvideos = getprocessedvideos(videos_out)
processedvideos.head()

found existing processedvideos.xlsx


,Unnamed: 0.1,Unnamed: 0,VideoID,ChildID,JokeType,JokeNum,JokeRep,JokeTake,HowFunny,LaughYesNo,...,Duration,Keypoints.when,Keypoints.file,Audio.when,Audio.file,Cleaned.when,Cleaned.file,LastError,Speech.file,Speech.when
0,0,0,2UWdXP.joke1.rep2.take1.Peekaboo.mp4,2UWdXP,Peekaboo,1,2,1,Slightly funny,No,...,15.175982,2023-09-19 11:39:38,..\data\1_interim\\2UWdXP.joke1.rep2.take1.Pee...,2023-09-19 12:31:14,..\data\1_interim\\2UWdXP.joke1.rep2.take1.Pee...,NaN,NaN,NaN,..\data\1_interim\2UWdXP.joke1.rep2.take1.Peek...,2023-09-20 16:58:38
1,1,1,2UWdXP.joke1.rep3.take1.Peekaboo.mp4,2UWdXP,Peekaboo,1,3,1,Slightly funny,No,...,10.585630,2023-09-19 11:40:12,..\data\1_interim\\2UWdXP.joke1.rep3.take1.Pee...,2023-09-19 12:31:15,..\data\1_interim\\2UWdXP.joke1.rep3.take1.Pee...,NaN,NaN,NaN,..\data\1_interim\2UWdXP.joke1.rep3.take1.Peek...,2023-09-20 16:58:39
2,2,2,2UWdXP.joke2.rep1.take1.NomNomNom.mp4,2UWdXP,NomNomNom,2,1,1,Funny,No,...,7.174514,2023-09-19 11:40:36,..\data\1_interim\\2UWdXP.joke2.rep1.take1.Nom...,2023-09-19 12:31:16,..\data\1_interim\\2UWdXP.joke2.rep1.take1.Nom...,NaN,NaN,NaN,..\data\1_interim\2UWdXP.joke2.rep1.take1.NomN...,2023-09-20 16:58:40
3,3,3,2UWdXP.joke2.rep2.take1.NomNomNom.mp4,2UWdXP,NomNomNom,2,2,1,Slightly funny,No,...,6.824348,2023-09-19 11:41:00,..\data\1_interim\\2UWdXP.joke2.rep2.take1.Nom...,2023-09-19 12:31:16,..\data\1_interim\\2UWdXP.joke2.rep2.take1.Nom...,NaN,NaN,NaN,..\data\1_interim\2UWdXP.joke2.rep2.take1.NomN...,2023-09-20 16:58:40
4,4,4,2UWdXP.joke2.rep3.take1.NomNomNom.mp4,2UWdXP,NomNomNom,2,3,1,Slightly funny,No,...,9.350991,2023-09-19 11:41:34,..\data\1_interim\\2UWdXP.joke2.rep3.take1.Nom...,2023-09-19 12:31:17,..\data\1_interim\\2UWdXP.joke2.rep3.take1.Nom...,NaN,NaN,NaN,..\data\1_interim\2UWdXP.joke2.rep3.take1.NomN...,2023-09-20 16:58:48


In [13]:
forceaudio = True
#output_ext="mp3"
output_ext="wav"

for index, r in processedvideos.iterrows():
    if forceaudio or pd.isnull(r["Audio.file"]):
        audiopath = convert_video_to_audio_moviepy(videos_in,r["VideoID"], videos_out, output_ext=output_ext)
        r["Audio.file"] = audiopath
        r["Audio.when"] = time.strftime("%Y-%m-%d %H:%M:%S", time.gmtime())
        #update this row in processedvideos dataframe
        processedvideos.loc[index] = r
        

processedvideos.to_excel(videos_out + "\\processedvideos.xlsx")
processedvideos.head()

MoviePy - Writing audio in ..\data\1_interim\\2UWdXP.joke1.rep2.take1.Peekaboo.wav


MoviePy - Done.
MoviePy - Writing audio in ..\data\1_interim\\2UWdXP.joke1.rep3.take1.Peekaboo.wav


MoviePy - Done.
MoviePy - Writing audio in ..\data\1_interim\\2UWdXP.joke2.rep1.take1.NomNomNom.wav


MoviePy - Done.


MoviePy - Writing audio in ..\data\1_interim\\2UWdXP.joke2.rep2.take1.NomNomNom.wav


...

MoviePy - Writing audio in ..\data\1_interim\\6c6MZQ.joke4.rep3.take1.ThatsNotACat.wav


MoviePy - Done.
MoviePy - Writing audio in ..\data\1_interim\\6c6MZQ.joke5.rep1.take1.TearingPaper.wav


MoviePy - Done.
MoviePy - Writing audio in ..\data\1_interim\\6c6MZQ.joke5.rep3.take1.TearingPaper.wav


MoviePy - Done.


,Unnamed: 0.1,Unnamed: 0,VideoID,ChildID,JokeType,JokeNum,JokeRep,JokeTake,HowFunny,LaughYesNo,...,Duration,Keypoints.when,Keypoints.file,Audio.when,Audio.file,Cleaned.when,Cleaned.file,LastError,Speech.file,Speech.when
0,0,0,2UWdXP.joke1.rep2.take1.Peekaboo.mp4,2UWdXP,Peekaboo,1,2,1,Slightly funny,No,...,15.175982,2023-09-19 11:39:38,..\data\1_interim\\2UWdXP.joke1.rep2.take1.Pee...,2023-09-22 14:03:28,..\data\1_interim\\2UWdXP.joke1.rep2.take1.Pee...,NaN,NaN,NaN,..\data\1_interim\2UWdXP.joke1.rep2.take1.Peek...,2023-09-20 16:58:38
1,1,1,2UWdXP.joke1.rep3.take1.Peekaboo.mp4,2UWdXP,Peekaboo,1,3,1,Slightly funny,No,...,10.585630,2023-09-19 11:40:12,..\data\1_interim\\2UWdXP.joke1.rep3.take1.Pee...,2023-09-22 14:03:28,..\data\1_interim\\2UWdXP.joke1.rep3.take1.Pee...,NaN,NaN,NaN,..\data\1_interim\2UWdXP.joke1.rep3.take1.Peek...,2023-09-20 16:58:39
2,2,2,2UWdXP.joke2.rep1.take1.NomNomNom.mp4,2UWdXP,NomNomNom,2,1,1,Funny,No,...,7.174514,2023-09-19 11:40:36,..\data\1_interim\\2UWdXP.joke2.rep1.take1.Nom...,2023-09-22 14:03:29,..\data\1_interim\\2UWdXP.joke2.rep1.take1.Nom...,NaN,NaN,NaN,..\data\1_interim\2UWdXP.joke2.rep1.take1.NomN...,2023-09-20 16:58:40
3,3,3,2UWdXP.joke2.rep2.take1.NomNomNom.mp4,2UWdXP,NomNomNom,2,2,1,Slightly funny,No,...,6.824348,2023-09-19 11:41:00,..\data\1_interim\\2UWdXP.joke2.rep2.take1.Nom...,2023-09-22 14:03:30,..\data\1_interim\\2UWdXP.joke2.rep2.take1.Nom...,NaN,NaN,NaN,..\data\1_interim\2UWdXP.joke2.rep2.take1.NomN...,2023-09-20 16:58:40
4,4,4,2UWdXP.joke2.rep3.take1.NomNomNom.mp4,2UWdXP,NomNomNom,2,3,1,Slightly funny,No,...,9.350991,2023-09-19 11:41:34,..\data\1_interim\\2UWdXP.joke2.rep3.take1.Nom...,2023-09-22 14:03:30,..\data\1_interim\\2UWdXP.joke2.rep3.take1.Nom...,NaN,NaN,NaN,..\data\1_interim\2UWdXP.joke2.rep3.take1.NomN...,2023-09-20 16:58:48
